In [ ]:
data = arrow::read_feather('/Users/sauterj1/Documents/Masters_Thesis/Figures_code/data/M1_Jake_Masters_thesis_cohort_analysis_matrix.feather')

In [ ]:
library(magrittr)

data$leiden_clusters %>% 
  unique() %>% unlist() %>% sort() %>% 
  paste0(collapse = ', ') %>% c()

In [ ]:
library(dplyr)
library(tidyr)
library(arrow)
library(cowplot)
library(ggplot2)
library(magrittr)
library(patchwork)

In [ ]:
data %>% head()

# How many patients per disease per flow cytometer

In [ ]:
data %>%
  group_by(anon_patient_id) %>%
  sample_n(1) %>% 
  ungroup() %>% 
  select(anon_patient_id, major_disease, cytometer_used) %>% 
  count(cytometer_used, major_disease)

## 187 Patients, 40k cells each

In [ ]:
data %>% count(anon_patient_id) %>% mutate(n = scales::comma(n)) %>% .$n %>% table()

## 4 Major Disease Classes

In [ ]:
data %>% count(major_disease) %>% mutate(n = scales::comma(n))

## Many minor disease classes

In [ ]:
data %>% count(minor_disease) %>% mutate(n = scales::comma(n))

## 28 Leiden Communities of Varying Size

In [ ]:
data %>% count(leiden_clusters) %>% arrange(desc(n)) %>%  mutate(n = scales::comma(n))

In [ ]:
library(stringr)

In [ ]:
unique_cell_fates <- data$cell_trajectories %>% .[!str_detect(., '_')] %>% unique()
cat('\n', length(unique_cell_fates), ' unique cell trajectories / fates found: ', unique_cell_fates, '\n\n')

## Leiden Clusters by Disease Class

In [ ]:
data$leiden_clusters %>% unique() %>% sort()

In [ ]:
leiden_communities <- 
  data$leiden_clusters

total_cells_per_disease <- 
  data$major_disease %>% 
  table()

cell_count_per_cluster <- 
  data %>% 
  mutate(leiden_clusters = as.integer(leiden_clusters)) %>% 
  count(leiden_clusters) %>% 
  mutate(n = format(round(as.numeric(n), 1), big.mark=","))

total_cells_per_cluster <- 
  data %>% 
  mutate(leiden_clusters = as.integer(leiden_clusters)) %>% 
  count(leiden_clusters) %>% 
  .$n

max_leiden_clusters <- 
  data$leiden_clusters %>% 
  as.integer() %>% 
  max()

num_diseases <- 
  data$major_disease %>% 
  unique() %>% 
  length()


diseases <-
  unique(data$major_disease)

disease_levels <-
  diseases %>% 
  str_length() %>%
  order() %>% rev() %>% diseases[.]

p1 <- 
  data%>% 
  mutate(major_disease = factor(major_disease, levels = disease_levels)) %>% 
  count(leiden_clusters, major_disease) %>% 
  rowwise() %>% 
  tidyr::complete(leiden_clusters, major_disease) %>% 
  rowwise() %>% 
  mutate(n = n / total_cells_per_cluster[leiden_clusters]) %>%
  # mutate(n = n / total_cells_per_disease[major_disease]) %>%
  ungroup() %>% 
  ggplot() + 
  geom_tile(aes(x = major_disease,
                y = leiden_clusters,
                fill = n)) +
  scale_fill_distiller(palette = "RdBu", 
                       direction = -1, 
                       na.value = '#0066b2', 
                       name = '% of Cluster\nComposed of\nby Disease') + 
  geom_text(data = cell_count_per_cluster,
            aes(x = num_diseases+.25,
                y = leiden_clusters,
                label = n),
            size = 2,
            hjust = -1,
            angle = 0,
            fontface = 'italic') +
  # geom_text(data = substantial_patients_df,
  #           aes(x = major_disease,
  #               y = leiden_clusters,
  #               label = n_substantial_patients),
  #           size = 2,
  #           hjust = 0.5,
  #           angle = 0,
  #           fontface = 'italic') +
  xlab('') + ylab('Leiden Community') +
  theme(axis.text.x = element_text(angle = -25,
                                   hjust = 0, 
                                   vjust = 1),
        axis.text.y.left = element_text(hjust = 1, 
                                        size = 5),
        panel.background = element_blank(),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(), 
        legend.title = element_text(size = 8), 
        legend.text = element_text(size = 6, 
                                   vjust = 1)) +
  coord_cartesian(ylim=c(0, max_leiden_clusters+1),
                  xlim=c(1.05, num_diseases+1.5)) +
  scale_y_continuous(breaks = 1:max_leiden_clusters) + 
  annotate(y=0.45, yend=0.45,
           x=0.5, xend=num_diseases+1,
           lwd=0.85, geom="segment")

print('Plotting ...')

ggsave(filename = 'Masters_Leiden_vs_Major_Disease.png', 
       plot = p1, width = 7, height = 10)

print('Completed')

## Leiden Clusters by Cell Type Heat Map

In [ ]:
cell_labels <- 
  data$cell_label %>% 
  unique()

cell_label_levels <-
  cell_labels %>%
  str_length() %>% 
  order() %>% rev() %>% 
  cell_labels[.]

p2 <- 
  data%>% 
  filter(cell_label != '') %>% 
  mutate(cell_label = factor(cell_label, levels = cell_label_levels)) %>%
  count(leiden_clusters, cell_label) %>% 
  rowwise() %>% 
  tidyr::complete(leiden_clusters, cell_label) %>% 
  rowwise() %>% 
  mutate(n = n / total_cells_per_cluster[leiden_clusters]) %>%
  # mutate(n = n / total_cells_per_disease[cell_label]) %>%
  ungroup() %>% 
  ggplot() + 
  geom_tile(aes(x = cell_label,
                y = leiden_clusters,
                fill = n)) +
  scale_fill_distiller(palette = "RdBu", 
                       direction = -1, 
                       na.value = '#0066b2', 
                       name = '% of Cluster\nComposed of\nby Cell Type') + 
  # scale_y_discrete(breaks = 1:length(unique(data$leiden_clusters))) +
  xlab('') + ylab('Leiden Community') +
  theme(axis.text.x = element_text(angle = -25,
                                   hjust = 0, 
                                   vjust = 1),
        # axis.text.y.left = element_text(hjust = 1, 
        #                                 size = 5),
        panel.background = element_blank(),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(), 
        legend.title = element_text(size = 8), 
        legend.text = element_text(size = 6, 
                                   vjust = 1)) +
  coord_cartesian(ylim=c(0, max_leiden_clusters+2),
                  xlim=c(1.05, num_diseases+0.5)) +
  scale_y_continuous(breaks = 1:max_leiden_clusters) + 
  annotate(y=0.45, yend=0.45,
           x=0.5, xend=num_diseases+0.5,
           lwd=0.85, geom="segment")

print('Plotting ...')

ggsave(filename = 'Masters_Leiden_vs_Cell_Type.png', 
       plot = p2, width = 7, height = 10)

print('Completed')

In [ ]:
library(patchwork)

p <- p2 + p1

theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  p + plot_annotation(caption = 'Heatmaps depicting Encrichment of cell type and disease type in each of the calculated Leiden communities.', 
                      theme = theme_border)

ggsave('Leiden_heatmaps.png', plot = p, width = 10, height = 7)

print('Done')

In [ ]:
# total_cells_per_disease <- 
#   data$cell_label %>% 
#   table()

# patient_count_per_disease <- 
#   data %>% 
#   count(cell_label) %>% 
#   mutate(n = n / 100e3) %>% 
#   mutate(n = format(round(as.numeric(n), 1), big.mark=","))

# cell_count_per_cluster <- 
#   data %>% 
#   mutate(leiden_clusters = as.integer(leiden_clusters)) %>% 
#   count(leiden_clusters) %>% 
#   mutate(n = format(round(as.numeric(n), 1), big.mark=","))

# total_cells_per_cluster <- 
#   data %>% 
#   mutate(leiden_clusters = as.integer(leiden_clusters)) %>% 
#   count(leiden_clusters) %>% 
#   .$n

# max_leiden_clusters <- 
#   data$leiden_clusters %>% 
#   as.integer() %>% 
#   max()

# num_diseases <- 
#   data$cell_label %>% 
#   unique() %>% 
#   length()



  # geom_text(data = patient_count_per_disease,
  #           aes(x = cell_label,
  #               y = max_leiden_clusters + 1.5, 
  #               label = n),
  #           angle = 0, 
  #           size = 3,
  #           fontface = 'italic') +
  # geom_text(data = cell_count_per_cluster,
  #           aes(x = num_diseases+.25,
  #               y = leiden_clusters,
  #               label = n),
  #           size = 2,
  #           hjust = -1,
  #           angle = 0,
  #           fontface = 'italic') +
  # geom_text(data = substantial_patients_df,
  #           aes(x = cell_label,
  #               y = leiden_clusters,
  #               label = n_substantial_patients),
  #           size = 2,
  #           hjust = 0.5,
  #           angle = 0,
  #           fontface = 'italic') +

## Leiden Communities on UMAP

In [ ]:
rep(5, n)

In [ ]:
colors

In [ ]:
require(RColorBrewer)
n <- 7
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))
colors[1:n]

colors <- scales::alpha(colors, alpha = 0.25)


ggplot() + 
  geom_bar(aes(y = rep(5, n), 
               fill = colors[1:n]))

colors[1:n]

In [ ]:
plot_umap <- function(points,
                      labels,
                      colors = NULL, 
                      alpha = 0.85, 
                      plot_labels = FALSE, 
                      plot_legend = FALSE, 
                      pt.cex = 0.005, 
                      legend.cex = 2.2, 
                      title = NA) {
  
  pad=0.2
  pch=19
  
  if (!is.factor(labels)) {
    new_levels = sort(unique(labels))
    labels = factor(labels, new_levels)
  }
  
  if (is.null(colors)) {
    require(RColorBrewer)
    n <- length(levels(labels))
    qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
    colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))
    trans_colors <- scales::alpha(colors, alpha = alpha)
  } else {
    trans_colors <- colors
  }

  xylim = range(points)
  xylim = xylim + ((xylim[2]-xylim[1])*pad)*c(-0.5, 0.5)
  
  # ps is pointsize of text only, not symbols or points
  par(mar=c(0.7,0.7,5,0.7), ps=10)
  plot(xylim, xylim, type="n", axes=F, frame=F)
  rect(xylim[1], xylim[1], xylim[2], xylim[2], border="#aaaaaa", lwd=0.25)
  
  
  
  points(points[,1], points[,2], col=trans_colors[as.integer(labels)],
         cex=pt.cex, pch=pch)
  
  
  num_levels <- length(levels(labels))
  label_xs <- vector('numeric', num_levels)
  label_ys <- vector('numeric', num_levels)
  
  i <- 0
  for (label in levels(labels)) {
    i <- i + 1
    label = as.integer(label)
    label_xs[i] <- median(points[labels == label, 1])
    label_ys[i] <- median(points[labels == label, 2])
  }
  
  
  if (isTRUE(plot_labels)) {
    text(label_xs, 
         label_ys, 
         levels(labels), 
         cex = 2, 
         font = 1)
    }
  
  if (isTRUE(plot_legend)) {
    labels.u = levels(labels)
    legend.pos = "topleft"
    legend.text = as.character(labels.u)

    legend(legend.pos, legend=legend.text, inset=0.03,
           col=colors[seq_along(labels.u)],
           bty="n", pch=pch, cex = legend.cex, pt.cex = 3.2)
  }
  
  
  if (!is.na(title)) {
      title(main = list(title, cex = 5))
  }
}

In [ ]:
labels <- data$leiden_clusters
labels <- factor(labels)

print('Plotting ...')


png('Umap_colored_by_leiden.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels, 
          alpha = 0.25, 
          plot_labels = TRUE, 
          plot_legend = FALSE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)

dev.off()

print('Completed')

## Cytometer on UMAP

In [ ]:
labels <- data$cytometer_used
labels <- factor(labels)

print('Plotting ...')

png('Umap_colored_by_cytometer.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels, 
          alpha = 0.25, 
          plot_labels = TRUE, 
          plot_legend = FALSE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)

dev.off()

print('Completed')

## One cytometer colored per plot

In [ ]:
umap_points <- 
  data %>% 
  select(UMAP1, UMAP2) %>% 
  as.data.frame()


labels <- data$cytometer_used
labels <- factor(labels)

require(RColorBrewer)
n <- length(levels(labels))
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
my_colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))

for (i in seq_along(levels(labels))) {
  
  label <- levels(labels)[i]
  cytometer_name <- as.character(label)
  
  print(cytometer_name)
  
  png(paste0('Umap_colored_by_cytometer_', cytometer_name, '.png'), 
      width = 800*2, 
      height = 500*2)
      
  current_labels <- ifelse(labels == label, cytometer_name, 'Other')
  current_labels %<>% factor(levels = c(cytometer_name, 'Other'))
  
  plot_umap(umap_points, 
            labels = current_labels, 
            colors = c(my_colors[i], scales::alpha('gray', 0.1)), 
            alpha = 1, 
            plot_labels = FALSE, 
            plot_legend = TRUE, 
            pt.cex = 0.005, 
            legend.cex = 2.2, 
            title = cytometer_name)

  dev.off()

  print('Completed')
  
}

# Gathering them all

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Canto 4.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Canto 5.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


g3 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Canto 6.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g4 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Fortessa 1.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g5 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Fortessa 2.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g6 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Symphony 2.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g7 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer_Symphony 3.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))



p1 <- ggplot2::ggplot() + ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf)
p2 <- ggplot2::ggplot() + ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf)
p3 <- ggplot2::ggplot() + ggplot2::annotation_custom(g3, -Inf, Inf, -Inf, Inf)
p4 <- ggplot2::ggplot() + ggplot2::annotation_custom(g4, -Inf, Inf, -Inf, Inf)
p5 <- ggplot2::ggplot() + ggplot2::annotation_custom(g5, -Inf, Inf, -Inf, Inf)
p6 <- ggplot2::ggplot() + ggplot2::annotation_custom(g6, -Inf, Inf, -Inf, Inf)
p7 <- ggplot2::ggplot() + ggplot2::annotation_custom(g7, -Inf, Inf, -Inf, Inf)


library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  list(p1, p2, p3, p4, p5, p6, p7) %>%
  patchwork::wrap_plots(ncol = 3) + 
  plot_annotation(caption = '')

ggsave('Patchwork_Single_Cytometer_UMAPs.png', plot = p, 
       height = 12, width = 17)

print('Done')

## Cell Type on UMAP

In [ ]:
labels <- data$cell_label
labels[labels == ""] = "Ungated"

print('Plotting ...')

png('Umap_colored_by_cell_type.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels, 
          alpha = 0.35, 
          pt.cex = 0.01, 
          plot_labels = FALSE, 
          plot_legend = TRUE)

dev.off()

print('Completed')

## Disease Type on UMAP

In [ ]:
labels <- data$major_disease

print("Plotting ...")

png('Umap_colored_by_Cancer_Type.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels, 
          alpha = 0.25, 
          pt.cex = 0.005)

dev.off()

print('Completed')

In [ ]:
data %>% filter(cell_labels == 'CD34+') %>% count(leiden_clusters) %>% arrange(desc(n)) %>% mutate(n = scales::comma(n))

In [ ]:
?readPNG

## Aggregating UMAPs into Panel figure

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cell_type.png') %>% 
  .[50:(nrow(.)-50),50:(ncol(.)-100),] %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_leiden.png') %>% 
  .[50:(nrow(.)-50),50:(ncol(.)-100),] %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))
g3 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_cytometer.png') %>% 
  .[50:(nrow(.)-50),50:(ncol(.)-100),] %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g4 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_Cancer_Type.png') %>% 
  .[50:(nrow(.)-50),50:(ncol(.)-100),] %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

p1 <- ggplot2::ggplot() + ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf)
p2 <- ggplot2::ggplot() + ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf)
p3 <- ggplot2::ggplot() + ggplot2::annotation_custom(g3, -Inf, Inf, -Inf, Inf)
p4 <- ggplot2::ggplot() + ggplot2::annotation_custom(g4, -Inf, Inf, -Inf, Inf)


library(patchwork)


theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))


p <- 
  (p1 + p2) / (p3 + p4) + 
   plot_annotation(caption = 'UMAP Plots depicting cells of all 40,000 cells from 187 patients colored by various labels.', 
                   theme = theme_border)

ggsave('Patchwork_UMAPs.png', plot = p, 
       height = 7, width = 10)

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/cds_30_neighbors_umap_300_center_principal_points.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Vertex_80_bifurcation_analysis.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


p1 <- ggplot2::ggplot() + ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf)
p2 <- ggplot2::ggplot() + ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf)


library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  (p2 / p1) +
  plot_layout(heights = c(1.6, 1)) + 
  plot_annotation(caption = 'Breakdown of cells before and after bifurcation at node Y_80 by cancer type and cell type.', 
                 theme = theme_border)

ggsave('Patchwork_Monocle_Trajectory_Breakdown.png', plot = p, 
       height = 12, width = 10)

print('Done')

## Flow Cytometry Background Figure

In [ ]:
data %>% head()

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Cytometer1.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Cytometer2.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


p1 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf) + 
  labs(caption = 'Diagram of a microfulidic flow cytometer nozzle. \nAdapted from Vembadi A, Menachery A, Qasaimeh MA. \nCell Cytometry: Review and Perspective on Biotechnological Advances. Front Bioeng Biotechnol. 2019;7: 147.') + 
  theme(plot.caption = element_text(size = 7))



p2 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf) + 
  labs(caption = 'Diagram of a flow cytometer with multiple Photo-Multiplier Tubes (PMTs). \nFair use under the Creative Commons License, attribution to: Kierano (https://commons.wikimedia.org/wiki/File:Cytometer.svg), \n„Cytometer“, https://creativecommons.org/licenses/by/3.0/legalcode') + 
  theme(plot.caption = element_text(size = 7))




library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  (p1 / p2) +
  plot_layout(heights = c(1.2, 2)) + 
  plot_annotation(caption = '', 
                 theme = theme_border)

ggsave('Patchwork_Cytometer.png', plot = p, 
       height = 8, width = 7)

print('Done')

In [ ]:
atribution_string = "Saeys Y, Van Gassen S, Lambrecht BN. Computational flow cytometry: helping to make sense of high-dimensional immunology data. \nNat Rev Immunol. 2016;16: 449–462."


library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Manual_Gating.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


p1 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf) + 
  theme(plot.caption = element_text(size = 7))




library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  p1 +
  plot_annotation(caption = paste0('Standard workflow of manual gating analysis in clinical flow cytometry.\n', atribution_string), 
                 theme = theme_border)

ggsave('Patchwork_Manual_Gating.png', plot = p, 
       height = 5, width = 8)

print('Done')

## Hematopoeisis Background Figure

In [ ]:
attribution_string = "By A. RadVector: RexxS, Mikael Häggström and birdy and Mikael Häggström, M.D. \nMikael Häggström - Own work based on: Hematopoiesis (human) diagram.svg, CC BY-SA 3.0, \nhttps://commons.wikimedia.org/w/index.php?curid=9420824"



library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Hematopoiesis_(human)_diagram_en.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


p1 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf) + 
  theme(plot.caption = element_text(size = 7))




library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  p1 +
  plot_annotation(caption = paste0('Hematopoietic Tree.\n', attribution_string), 
                 theme = theme_border)

ggsave('Patchwork_Heme_Tree.png', plot = p, 
       height = 10, width = 11, bg = 'white')

print('Done')

In [ ]:
paste0('Standard workflow of manual gating analysis in clinical flow cytometry.\n', atribution_string)

## Sample Measurement Date and Cytometer Heatmap

In [ ]:
#!/bin/Rscript

library(dplyr)
library(ggplot2)
library(stringr)
library(magrittr)
library(flowCore)
library(flowStats)
library(patchwork)
library(lubridate)
library(stringr)
library(knitr)

filter <- dplyr::filter

### Input data
patient_df <- 
  read.csv('~/Documents/Woodlist/Input_Flow_DFs/jake_processed_flow_df_ids_102021_active.csv', 
           stringsAsFactors = FALSE) 

patient_df <- 
  patient_df %>% 
  filter(Exclusion.Reason == "Not Excluded")

patient_df %>% dim()

In [ ]:
m1_meta_info_list <- vector('list', nrow(patient_df))

parse_meta_info <- function(parsed_meta) {
  baseline_date      <- parsed_meta$`CST BASELINE DATE` %>% 
    str_split('T') %>% .[[1]] %>% .[1]

  setup_date         <- parsed_meta$`CST SETUP DATE` %>% 
    str_split('T') %>% .[[1]] %>% .[1]
  
  config_create_date <- parsed_meta$`CYTOMETER CONFIG CREATE DATE` %>% 
    str_split('T') %>% .[[1]] %>% .[1]
  
  beads_lot_id       <- parsed_meta$`CST BEADS LOT ID`
  config_name        <- parsed_meta$`CYTOMETER CONFIG NAME`
  
  cytometer <- parsed_meta$`$CYT`
  cyt_num   <- parsed_meta$CYTNUM
  date      <- parsed_meta$`$DATE`
  
  param_names <- 
    colnames(parsed_meta$SPILL)
  
  # marker names
  
  marker_names <- 
    names(parsed_meta) %>% 
    str_detect('P[0-9]+S') %>% 
    parsed_meta[.] %>% 
    unname() %>% unlist() %>% 
    sort()
  
  meta_info_list <- 
    list(
      cytometer = cytometer, 
      cyt_num = cyt_num, 
      date = date, 
      baseline_date = baseline_date, 
      setup_date = setup_date, 
      config_create_date = config_create_date, 
      beads_lot_id = beads_lot_id, 
      config_name = config_name, 
      param_names = param_names, 
      marker_names = marker_names
    ) 
  
  meta_info_list
}

In [ ]:
patient_df[, 'Flow.Data.Folder'] %>% sapply(function(dir) dir.exists(dir)) %>% {.[which(. == FALSE)]}

In [ ]:
patient_df <- 
  patient_df %>% filter(Accession.Number != "F17-8882")

In [ ]:
for (i in seq_len(nrow(patient_df))) {
  
  flow_directory <- patient_df[i, "Flow.Data.Folder"]
  
  cat('Processing flow directory:', i, 'out of:', nrow(patient_df))
  cat('\nFlow directory: ', flow_directory, '\n\n')
  
  m1_fcs_file <- 
    patient_df[i, "M1_path"]
  
  m1_fcs_file <- 
    m1_fcs_file %>% 
    stringr::str_split(',') %>%
    .[[1]] %>% .[1]
  
  
  parsed_meta <- description(read.FCS(m1_fcs_file, 
                                      transformation = FALSE, 
                                      which.lines = 1))
  
  
  m1_meta_info_list[[i]] <- parse_meta_info(parsed_meta)

}

m1_meta_info_df <- 
  do.call(rbind, m1_meta_info_list) %>% 
  as.data.frame() %>% 
  mutate(
    date = lubridate::dmy(date)) %>% 
  mutate(baseline_date = lubridate::ymd(baseline_date), 
         setup_date = lubridate::ymd(setup_date), 
         flow_id = patient_df$Accession.Number) %>% 
  arrange(cyt_num, date, baseline_date, setup_date)


m1_meta_info_df %>% head()

In [ ]:
cytometer_range <- function(df, cytom_num) {
  df <- 
    df %>% 
    arrange(date) %>% 
    filter(cyt_num == cytom_num)
  
  first <- 
    df %>% 
    head(1) %>% 
    .$date
  
  last <- 
    df %>% 
    tail(1) %>% 
    .$date

  c(first, last)
}

cytometer_ids <- 
  m1_meta_info_df %>% 
  .$cyt_num %>% 
  unique() %>% 
  unlist()


# 8 different cytometers used for m1 over the years

sorted_meta_info_df <- 
  m1_meta_info_df %>% 
  mutate(year = year(date), 
         cyt_num = as.character(cyt_num), 
         cytometer = as.character(cytometer)) %>% 
  count(cyt_num, year) %>% 
  arrange(year)
         
# Starts with V -- FACSCanto
# Starts with R -- LSRFortessa
unique_cytometers_used <- 
  sorted_meta_info_df$cyt_num %>% 
  unique()


# same three used from 2016-2019 (21, 98, 99)
# new machines introduced in 2020 and 2021


# make plot -- time on x axis, machine on y axis, 
# heatmap value of how many uses

library(cowplot)

p1 <-
  m1_meta_info_df %>% 
  mutate(date = floor_date(date, 'quarter'),  
         cyt_num = as.character(cyt_num),
         cytometer_name =
          case_when(
            cyt_num == "V657338000021" ~ "CANTO IV",
            cyt_num == "V657338000098" ~ "CANTO V",
            cyt_num == "V657338000099" ~ "CANTO VI",
            cyt_num == "R658222R1012" ~ "Fortessa I",
            cyt_num == "R65822R1018"  ~ "Fortessa II",
            cyt_num == "R66093700072" ~ "Symphony1",
            cyt_num == "R66093700081" ~ "Symphony2",
            cyt_num == "R66093700082" ~ "Symphony3"
          )) %>% 
  mutate(cytometer_name = factor(cytometer_name, levels = rev(sort(unique(cytometer_name))))) %>% 
  count(cytometer_name, date) %>% 
  arrange(date) %>% 
  ggplot() + 
  geom_tile(aes(x = date, y = cytometer_name, fill = n)) + 
  geom_text(aes(x = date, y = cytometer_name, label = n)) + 
  scale_fill_distiller(palette = 'Reds', 
                       direction = 1) + 
  labs(x = 'Year (Divided into Quarters)', y = 'Flow Cytometer') + 
  guides(fill = guide_legend(title = 'Num. Patients')) +
  ggtitle("Cytometers Used Over Time") + 
  theme_cowplot()

library(patchwork)



theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p2 <- 

p1 <- 
  p1 + plot_annotation(caption = 'Heatmap depicting which Flow Cytometer and Date the patients in this study were assessed on.', 
                      theme = theme_border)

ggsave('Full_Cohort_use_over_time.png', plot = p1, width = 10, height = 5)

print('Done')

In [ ]:
getwd()

# Frame any image and reprint to new filename

In [ ]:
library(png)
library(grid)
library(ggplot2)
library(stringr)
library(magrittr)
library(patchwork)

frame_image <- function(filename) {

  output_filename <- 
    filename %>%
    stringr::str_split('\\.') %>% 
    .[[1]] %>% .[1] %>% paste0('_with_border.png')
  
  print(output_filename)

  g <- 
    readPNG(filename)
  
  plot_height <- nrow(g) * 2
  plot_width <-  ncol(g) * 2
  
  g <- g %>% 
    rasterGrob(interpolate=TRUE, 
               width=ggplot2::unit(1,"npc"),
               height=ggplot2::unit(1,"npc"))
  
 
  p <- ggplot2::ggplot() + ggplot2::annotation_custom(g, -Inf, Inf, -Inf, Inf)
  
  theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))


  p <- 
    p + plot_annotation(caption = '', 
                        theme = theme_border)

  ggsave(output_filename, plot = p, 
         height = plot_height, 
         width = plot_width, 
         units = 'px')

  print(p)
  print('Saved')
  
  
}

In [ ]:
frame_image('~/Documents/Masters_Thesis/Figures_code/Thesis_Flowchart.png')

## Pre-processing figure

In [ ]:
library(flowCore)
library(ggplot2)
library(dplyr)
library(png)
library(grid)
library(patchwork)

fcs <- read.FCS('~/Documents/Woodlist/Validated_Diseased_Cohort_Anon_Flow_Folders/3371373/processed_M1.fcs', 
                transformation = FALSE)


p1 <- 
  data.frame(
    x = seq(-10, 10, .1), 
    y = asinh(seq(-10, 10, .1))) %>% 
    ggplot() + 
    geom_line(aes(x = x, 
                  y = y), 
              lwd = 1.5) + 
    geom_hline(yintercept = 0, 
               lty = 2) + 
    geom_vline(xintercept = 0, 
               lty = 2) + 
    xlab('Input Value') + ylab('Transformed Output Value') + 
    ggtitle('Hyperbolic Arc Sin Transform') 


p2 <- 
  exprs(fcs)[, "CD33"] %>% 
  {sinh(.) * 150} %>% 
  data.frame(x = .) %>% 
  ggplot() + 
  geom_histogram(aes(x = x), 
                 color = 'black', 
                 fill = 'skyblue') + 
  xlab('CD33 Expression') + 
  ylab('Number of Cells') + 
  xlim(c(0, 20e3)) + 
  ylim(c(0, 10e3)) + 
  ggtitle('c) CD33 Expr. Before Trans.') + 
  theme(plot.title = element_text(face = 'bold'))


p3 <- 
  exprs(fcs)[, "CD33"] %>% 
  data.frame(x = .) %>% 
  ggplot() + 
  geom_histogram(aes(x = x), 
                 color = 'black', 
                 fill = 'skyblue') + 
  xlab('Asinh Transformed CD33 Expression') + 
  ylab('Number of Cells') + xlim(c(-1, 6)) + 
  ggtitle('   CD33 Expr. After Trans.') + 
  theme(plot.title = element_text(face = 'bold'))



panel_1 <- p2 / p3



signals_aq_check <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/figures/Signal_aquisition_check.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc")) 


signals_aq_check <- 
  ggplot2::ggplot() + 
  ggplot2::annotation_custom(signals_aq_check, -Inf, Inf, -Inf, Inf) + 
  labs(title = 'a)') + 
  theme(plot.title = element_text(face = 'bold'))

compensation <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/figures/compensation.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

compensation <- 
    ggplot2::ggplot() + 
    ggplot2::annotation_custom(compensation, -Inf, Inf, -Inf, Inf) + 
  labs(title = 'b)', 
       caption = "Chattopadhyay PK, Hogerkorp C-M, Roederer M. \nA chromatic explosion: the development and future of multiparameter flow cytometry. \nImmunology. 2008;125: 441–449.") + 
  theme(plot.title = element_text(face = 'bold'), 
        plot.caption = element_text(size = 5))



singlets <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/figures/singlets.png') %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


singlets <- ggplot2::ggplot() + 
    ggplot2::annotation_custom(singlets, -Inf, Inf, -Inf, Inf) +
    labs(title = 'd)', 
         caption = 'Meya DB, Okurut S, Zziwa G, Cose S, Bohjanen PR, Mayanja-Kizza H, et al. \nMonocyte Phenotype and IFN-γ-Inducible Cytokine Responses Are Associated with Cryptococcal Immune Reconstitution Inflammatory Syndrome. \nJ Fungi (Basel). 2017;3. doi:10.3390/jof3020028') + 
    theme(plot.title = element_text(face = 'bold'), 
          plot.caption = element_text(size = 5))




theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))


p <- 
(
  (signals_aq_check | compensation) / 
  (panel_1  | singlets)
) + plot_annotation(caption = '', 
                theme = theme_border)

ggsave('Patchwork_preprocessing_panel.png', plot = p, height = 8, width = 10)


print('Completed')

## Marker Expression UMAPs

In [ ]:
plot_expression_umap <- function(data, expression_data, 
                                 marker = "CD19", 
                                 plot_label_positions = FALSE) {
  
  expression <- expression_data[, marker]
  
  expression <- 
    ifelse(expression > quantile(expression, .999), 
           quantile(expression, .999), 
           expression)
  
  expression <- 
    ifelse(expression < quantile(expression, .001), 
           quantile(expression, .001), 
           expression)
  
  midpoint <- 
    median(expression)
  
  p <-
    data.frame(
      x = data[, 1], 
      y = data[, 2], 
      expression = expression) %>% 
    ggplot() +
    geom_scattermore(mapping = aes(x = x, y = y, 
                                   col = expression)) +
    scale_color_gradient2(low = 'blue',
                          high = 'red',
                          midpoint = midpoint,
                          name = 'Expression Val') +
    ggtitle(paste0(marker)) + 
    xlab("UMAP 1") + 
    ylab("UMAP 2") + 
    theme(plot.title = element_text(size = 12, face = "bold"), 
          axis.title = element_text(size = 8, face = 'bold'), 
          legend.position = 'none') 
  
  return(p)
}

In [ ]:
# Plotting marker expression on umap with above function
library(png)
library(grid)
library(ggplot2)
library(stringr)
library(magrittr)
library(patchwork)
library(scattermore)

if (!exists('data')) {
  data <- 
    arrow::read_feather('/Users/sauterj1/Documents/Masters_Thesis/Figures_code/data/M1_Jake_Masters_thesis_cohort_analysis_matrix.feather')
}

row_key <- 
  data[, 'row_key']

marker_data <- 
  data[,colnames(data) %in% c('FSC-H', 'SSC-H', 'CD13', 'CD15', 'CD19', 'CD33', 'CD34', 'CD38', 'CD45', 'CD71', 'CD117', 'HLA-DR')]

umap_points <- data[, colnames(data) %in% c('UMAP1', 'UMAP2')]

plot_list <- 
  vector('list', ncol(marker_data))

i <- 0
for (colname in colnames(marker_data)) {
  i <- i + 1
  cat('Making plot for marker: ', colname, '\n')
  plot_list[[i]] <-
    plot_expression_umap(umap_points, 
                         marker_data, 
                         colname)
}

In [ ]:
p <- patchwork::wrap_plots(plot_list, ncol = 3)


theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75), 
        plot.caption = element_text(size = 14))

p <- 
  p + 
  plot_annotation(caption = 'Cell marker expression values (hyperbolic arcsine) shown on UMAP reduced dimensional representation of all patient cells.', 
                  theme = theme_border)




ggsave('Patchwork_expression_umaps.png', plot = p, width = 25, height = 16)

print('Done')

## Only color cells from leiden cluster (one that start cell came from)

In [ ]:
library(dplyr)

max_cd34_cluster <- 
  expression_data %>% 
  filter(cell_label == 'CD34+') %>% 
  count(leiden_clusters) %>% 
  arrange(desc(n)) %>% 
  head(1) %>% .$leiden_clusters

colors <- ifelse(expression_data$leiden_clusters == max_cd34_cluster, 'Start Cell Cluster', 'Other')

In [ ]:
plot_umap <- function(umap_points, labels, colors) {
  
  p <-
    data.frame(
      x = umap_points[, 1], 
      y = umap_points[, 2]) %>%
    ggplot() +
    geom_scattermore(mapping = aes(x = x, y = y, 
                     col = colors)) +
    scale_color_manual(values = c('Start Cell Cluster' = 'skyblue', 'Other' = '#D3D3D3')) +
    xlab("UMAP 1") + 
    ylab("UMAP 2") + 
    theme(plot.title = element_text(size = 12, face = "bold"), 
          axis.title = element_text(size = 8, face = 'bold')) + 
    ggtitle('Starting Cell Cluster Location (CD34+)')
  
  return(p)
}

In [ ]:
start_cell_cluster <- plot_umap(umap_points, expression_data$leiden_clusters, colors)

In [ ]:
plot_list[[4]] <- start_cell_cluster


p <- 
  wrap_plots(plot_list, 
             ncol = 2)

theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75), 
        plot.caption = element_text(size = 8))


p <- 
  p + 
  plot_annotation(caption = 'Signal value / marker expression for all cells shown on UMAP.', 
                  theme = theme_border)


png('Masters_cohort_marker_expression_on_UMAP.png', 
    width = 1000, height = 700)

print(p)
dev.off()

# Wrapping pre and post bifurcation

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Marker_expression_value_pre_and_post_bifurcation_violin_cd34+_only.png') %>% 
  {.[1:(nrow(.) - 20), , ]} %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


p1 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf) + 
  labs(caption = '') + 
  theme(plot.caption = element_text(size = 7), 
          plot.margin = unit(c(0,0,0,0), 'mm'))


library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- p1 + 
  plot_annotation(caption = 'Violin plots of marker expression values for all 10 markers and 2 scatter values assessed per single cell event.', 
                 theme = theme_border)

ggsave('Patchwork_violin.png', plot = p, 
       height = 12, width = 10)

print('Done')

## Leiden Resolution Adjustment Figure

In [ ]:
labels1 <- arrow::read_feather('./M1_Jake_Masters_thesis_cohort_random_40k_subsample_30_neighbors_Leiden_res_02_labels.feather')
labels2 <- arrow::read_feather('./M1_Jake_Masters_thesis_cohort_random_40k_subsample_30_neighbors_Leiden_res_035_100_iters_labels.feather')
labels3 <- arrow::read_feather('./M1_Jake_Masters_thesis_cohort_random_40k_subsample_30_neighbors_Leiden_res_07_labels.feather')

In [ ]:
plot_umap

In [ ]:
labels1 <- labels1 %>% unlist %>% as.integer() %>% {(. - min(.)) + 1}
labels2 <- labels2 %>% unlist %>% as.integer() %>% {(. - min(.)) + 1}
labels3 <- labels3 %>% unlist %>% as.integer() %>% {(. - min(.)) + 1}

In [ ]:
umap_points <- data[, c('UMAP1', 'UMAP2')]

png('leiden_umap_1.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels1, 
          alpha = 0.25, 
          plot_labels = FALSE, 
          plot_legend = FALSE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)

dev.off()

print('Done UMAP 1')

png('leiden_umap_2.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels2, 
          alpha = 0.25, 
          plot_labels = FALSE, 
          plot_legend = FALSE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)
dev.off()

print('Done UMAP 2')

png('leiden_umap_3.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels3, 
          alpha = 0.25, 
          plot_labels = FALSE, 
          plot_legend = FALSE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)
dev.off()

print('Done UMAP 3')

In [ ]:
png('leiden_umap_4.png', 
    width = 800*2, 
    height = 500*2)

plot_umap(umap_points, 
          labels = labels2, 
          alpha = 0.25, 
          plot_labels = TRUE, 
          plot_legend = TRUE, 
          pt.cex = 0.005, 
          legend.cex = 2.2)
dev.off()

print('Done UMAP 4')

In [ ]:
library(png)
library(grid)
library(magrittr)
library(patchwork)


g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/leiden_umap_1.png') %>% 
  {.[70:nrow(.)-70, 70:ncol(.)-70,]} %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/leiden_umap_2.png') %>% 
  {.[70:nrow(.)-70, 70:ncol(.)-70,]} %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g3 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/leiden_umap_3.png') %>% 
  {.[70:nrow(.)-70, 70:ncol(.)-70,]} %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g4 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/leiden_umap_4.png') %>% 
  {.[70:nrow(.)-70, 70:ncol(.)-70,]} %>% 
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))



p1 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf) + 
  labs(caption = '') + 
  ggtitle('Resolution 0.2') + 
  theme(plot.caption = element_text(size = 7), 
          plot.margin = unit(c(0,0,0,0), 'mm'))

p2 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf) + 
  ggtitle('Resolution = 0.35') + 
  labs(caption = '') + 
  theme(plot.caption = element_text(size = 7), 
          plot.margin = unit(c(0,0,0,0), 'mm'))

p3 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g3, -Inf, Inf, -Inf, Inf) + 
  ggtitle('Resolution = 0.7') + 
  labs(caption = '') + 
  theme(plot.caption = element_text(size = 7), 
          plot.margin = unit(c(0,0,0,0), 'mm'))

p4 <- ggplot2::ggplot() + 
  ggplot2::annotation_custom(g4, -Inf, Inf, -Inf, Inf) + 
  ggtitle('Resolution = 035; Communities labeled') + 
  labs(caption = '') + 
  theme(plot.caption = element_text(size = 7), 
          plot.margin = unit(c(1,0,0,0), 'mm'))


theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  (p1 + p2) / (p3 + p4) +
  plot_annotation(caption = 'Effect of adjusting Leiden community detection resolution shown visually\nusing the first two components UMAP dimensionality reduciton.', 
                 theme = theme_border)

ggsave('Patchwork_Leiden_Resolution.png', plot = p, 
       height = 9, width = 10)

print('Done')